#Opis zadania - Grafik dla sklepu
<b>Cel</b>: Minimalizacja wydatków.<br>
<b>Zmienne decyzyjne</b>: Jakiego dnia dana osoba ma przyjść.<br>
<b>Wymagania obsługi</b>:<br>

| Pon | Wto | Śro | Czw | Pią |
|-----|-----|-----|-----|-----|
|  2  |  1  |  1  |  1  |  3  |

<b>Dostępność pracowników</b>:<br>
Każda osoba może przepracować max. 3 dni.<br>

| Osoba    | Dostępność | Stawka [zł/d] |
|----------|------------|---------------|
| Ania     | Codziennie | 100 |
| Stefan   | Pon        | 50  |
| Hektor   | Pon - Śro  | 60  |
| Olaf     | Codziennie | 40  |
| Lidia    | Czw - Pią  | 110 |
| Piotr    | Pon - Śro  | 70  |

#Brudnopis

In [1]:
#plikxls = xlwt.Workbook(encoding="utf-8")
#arkuszZad = plikxls.add_sheet("WymogiObslugi")
#arkuszPrac = plikxls.add_sheet("Pracownicy")
#for n, dn in enumerate(DniTygodnia):
#    arkuszZad.write(0, n, dn)
#    arkuszZad.write(1, n, MinIlPrac[dn])
#
#arkuszPrac.write(0, 0, 'Pracownik')
#arkuszPrac.write(0, 1, 'Stawka')
#for nd, d in enumerate(DniTygodnia):
#    arkuszPrac.write(0, 2 + nd, 'Dostęp_' + d)
#for np, p in enumerate(Pracownicy):
#    arkuszPrac.write(np + 1, 0, p)
#    arkuszPrac.write(np + 1, 1, StawkaPrac[p])
#    for nd, d, in enumerate(DniTygodnia):
#        arkuszPrac.write(np + 1, nd + 2, DostepPrac[p][d])
#plikxls.save("SklepGrafik.xls")

#Instalacja bibliotek

In [2]:
!pip install pulp
!sudo apt-get install coinor-cbc glpk-utils coinor-clp
from pulp import *

Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [3]:
import xlwt
import pandas as pd
import pulp

#Dane z zadania

In [4]:
# Dane z zadania
DniTygodnia = [ 'pn', 'wt', 'śr', 'cz', 'pt']
MinIlPrac = { 'pn': 2, 'wt': 1, 'śr': 1, 'cz': 1, 'pt': 3 }
#MinIlPrac = { 'pn': 5, 'wt': 3, 'śr': 3, 'cz': 2, 'pt': 3 }
Pracownicy = ['Ania', 'Stefan', 'Hektor', 'Olaf', 'Lidia', 'Piotr']
DostepPrac = { 'Ania': { 'pn': 1, 'wt': 1, 'śr': 1, 'cz': 1, 'pt': 1},
              'Stefan': { 'pn': 1, 'wt': 0, 'śr': 0, 'cz': 0, 'pt': 0},
              'Hektor': { 'pn': 1, 'wt': 1, 'śr': 1, 'cz': 0, 'pt': 0},
              'Olaf': { 'pn': 1, 'wt': 1, 'śr': 1, 'cz': 1, 'pt': 1},
              'Lidia': { 'pn': 0, 'wt': 0, 'śr': 0, 'cz': 1, 'pt': 1},
              'Piotr': { 'pn': 1, 'wt': 1, 'śr': 1, 'cz': 0, 'pt': 0}}
StawkaPrac = { 'Ania': 100,
              'Stefan': 50,
              'Hektor': 60,
              'Olaf': 40,
              'Lidia': 110,
              'Piotr': 70 }
MaxCzasPracy = { 'Ania': 3,
              'Stefan': 3,
              'Hektor': 3,
              'Olaf': 3,
              'Lidia': 3,
              'Piotr': 3}

#Tworzenie arkusza kalkulacyjnego

In [5]:
with pd.ExcelWriter('SklepGrafikDane.xlsx', mode='w') as plikArkusza:
  pd.DataFrame(DostepPrac).transpose().to_excel(plikArkusza, sheet_name='DostepPracownikow')
  pd.DataFrame(MinIlPrac, index=range(1)).transpose().to_excel(plikArkusza, sheet_name='WymogiObslugi')
  pd.DataFrame(StawkaPrac, index=range(1)).transpose().to_excel(plikArkusza, sheet_name='StawkiPracownikow')
  pd.DataFrame(MaxCzasPracy, index=range(1)).transpose().to_excel(plikArkusza, sheet_name='LimitCzasuPracy')

# Rozwiązanie skomplikowane
Z wykorzystaniem kombinacji

In [6]:
# Zlicza pensję dla danej listy pracowników na podstawie ich stawek
def sumaPensji(pracownicy, stawki):
    return sum(stawki[p] for p in pracownicy)

# Pobranie arkuszy
ArkWymogi = pd.read_excel("SklepGrafikDane.xlsx", "WymogiObslugi", index_col=0)
ArkPlace = pd.read_excel("SklepGrafikDane.xlsx", "StawkiPracownikow", index_col=0)
ArkDostep = pd.read_excel("SklepGrafikDane.xlsx", "DostepPracownikow", index_col=0)
ArkCzasPracy = pd.read_excel("SklepGrafikDane.xlsx", "LimitCzasuPracy", index_col=0)
# Odczyt danych z arkuszy
Pracownicy = ArkDostep.index
DniTygodnia = ArkWymogi.index
WymogiObslugi = {d: ArkWymogi[0][d] for d in DniTygodnia}
LimitCzasuPracy = {p: ArkCzasPracy[0][p] for p in Pracownicy}
Place = {p: ArkPlace[0][p] for p in Pracownicy}
Dostepnosc = {p: {d: ArkDostep[d][p] for d in DniTygodnia} for p in Pracownicy}
# Kombinacje wszystkich grafików dzienych w liście dla całego tygodnia
MozliweGrafiki = [[c for c in pulp.allcombinations(Pracownicy, WymogiObslugi[d])
                      if len(c) >= WymogiObslugi[d]] for d in DniTygodnia]
# Wirtualna kartka do zaznaczania krzyżykiem wybranych grafików (każdy dzień na osobnej liście)
ListaWyboru = [pulp.LpVariable.dicts('grafik_' + d, MozliweGrafiki[nd],
                                     lowBound = 0, upBound = 1,
                                     cat = pulp.LpInteger)
                for nd, d in enumerate(DniTygodnia)]
# Definicja problemu
ProblemGrafiku = LpProblem("Grafik_sklepowy", LpMinimize)
# Definicja funkcji celu - wypłacanej pensji
ProblemGrafiku += lpSum([lpSum([ListaWyboru[nd][g] * sumaPensji(g, Place) for g in MozliweGrafiki[nd]]) for nd, d in enumerate(DniTygodnia)])
# Ograniczenie czasu pracy każdego pracownika
for p in Pracownicy:
    # Łączny czas pracy w tygodniu
    ProblemGrafiku += lpSum([
         [ListaWyboru[nd][g] * g.count(p) for g in MozliweGrafiki[nd]]
          for nd, d in enumerate(DniTygodnia)]) <= LimitCzasuPracy[p]
    # Dostępność w konkretne dni
    for nd, d in enumerate(DniTygodnia):
      for g in MozliweGrafiki[nd]:
        if p in g:
          ProblemGrafiku += ListaWyboru[nd][g] <= Dostepnosc[p][d]
# Możliwość wyboru tylko (i dokładnie) jednego grafiku dla danego dnia
for nd, d in enumerate(DniTygodnia):
    ProblemGrafiku += lpSum(ListaWyboru[nd]) == 1
# Rozwiązanie problemu liniowego
ProblemGrafiku.solve()
# Przygotowanie tabeli z grafikiem (spełnienie jednorodnego wymiaru)
Grafik = [[None] * max([WymogiObslugi[dni] for dni in DniTygodnia]) for d in DniTygodnia]
# Uzupełnienie tabeli wybranymi grafikami
for nd, d in enumerate(DniTygodnia):
    for g in MozliweGrafiki[nd]:
        if ListaWyboru[nd][g].value() == 1:
            Grafik[nd] = g
# Stworzenie ramki przygotowanej do pokazania
RamkaGrafiku = pd.DataFrame(Grafik,
                            index=DniTygodnia,
                            columns=range(1, max(WymogiObslugi.values()) + 1)
                            ).transpose()
# Dodanie arkusza 'Grafik' z wyznaczonym grafikiem do istniejącego pliku Excela
with pd.ExcelWriter('SklepGrafikDane.xlsx', mode='a') as plikArkusza:
    RamkaGrafiku.to_excel(plikArkusza, sheet_name='Grafik')
# Pokazanie wyniku na stronie
print(RamkaGrafiku)


       pn    wt      śr    cz     pt
1  Stefan  Olaf  Hektor  Olaf   Ania
2  Hektor  None    None  None   Olaf
3    None  None    None  None  Lidia


# Rozwiązanie prostsze
Z wykorzystaniem tabeli

In [7]:
# Pobranie arkuszy
ArkWymogi = pd.read_excel("SklepGrafikDane.xlsx", "WymogiObslugi", index_col=0)
ArkPlace = pd.read_excel("SklepGrafikDane.xlsx", "StawkiPracownikow", index_col=0)
ArkDostep = pd.read_excel("SklepGrafikDane.xlsx", "DostepPracownikow", index_col=0)
ArkCzasPracy = pd.read_excel("SklepGrafikDane.xlsx", "LimitCzasuPracy", index_col=0)
# Odczyt danych z arkuszy
Pracownicy = ArkDostep.index
DniTygodnia = ArkWymogi.index
WymogiObslugi = {d: ArkWymogi[0][d] for d in DniTygodnia}
LimitCzasuPracy = {p: ArkCzasPracy[0][p] for p in Pracownicy}
Place = {p: ArkPlace[0][p] for p in Pracownicy}
Dostepnosc = {p: {d: ArkDostep[d][p] for d in DniTygodnia} for p in Pracownicy}

# Krotki dzień tygodnia - pracownik
NaglowekGrafiku = [(d, p) for d in DniTygodnia for p in Pracownicy]
# Tabelka z zaznaczonymi krzyżykami
TabelaGrafiku = LpVariable.dicts("Obecnosc", [ng for ng in NaglowekGrafiku], cat=LpBinary)

# Definicja problemu
ProblemGrafiku = LpProblem("Grafik_sklepowy", LpMinimize)
# Definicja funkcji celu - wypłacanej pensji
ProblemGrafiku += lpSum([TabelaGrafiku[d, p] * Place[p] for d, p in NaglowekGrafiku])
# Spełnienie wymogu liczebności obsługi
for d in DniTygodnia:
    ProblemGrafiku += lpSum([TabelaGrafiku[d, p] for p in Pracownicy]) >= WymogiObslugi[d]
# Ograniczenie łącznego czasu pracy
for p in Pracownicy:
    ProblemGrafiku += lpSum([TabelaGrafiku[d, p] for d in DniTygodnia]) <= LimitCzasuPracy[p]
# Uwzględnienie niedostępności pracowników
for d, p in NaglowekGrafiku:
    ProblemGrafiku += TabelaGrafiku[d, p] <= Dostepnosc[p][d]
# Rozwiązanie problemu
ProblemGrafiku.solve()
# Pokazanie statusu rozwiązania
print(LpStatus[ProblemGrafiku.status])
# Stworzenie tabeli z zaznaczonym grafikiem
RamkaGrafiku = pd.DataFrame([
                             [TabelaGrafiku[d, p].value() for d in DniTygodnia]
                             for p in Pracownicy],
                            index=Pracownicy, columns=DniTygodnia)
# Pokazanie grafiku
print(RamkaGrafiku)
# Zapis grafiku do pliku lokalnego (dopisanie do już istniejącego)
with pd.ExcelWriter('SklepGrafikGotowy.xlsx', mode='r') as PlikArkusza:
    RamkaGrafiku.to_excel(PlikArkusza, 'Grafik')


Optimal
         pn   wt   śr   cz   pt
Ania    0.0  0.0  0.0  0.0  1.0
Stefan  1.0  0.0  0.0  0.0  0.0
Hektor  1.0  1.0  0.0  0.0  0.0
Olaf    0.0  0.0  1.0  1.0  1.0
Lidia   0.0  0.0  0.0  0.0  1.0
Piotr   0.0  0.0  0.0  0.0  0.0


#Podsumowanie efektów
Zmieniając dane w arkuszu można układać grafik dla dowolnych dni (zamiast pn mogłyby być daty).<br>
Wystarczy dodać wiersze z nowymi pracownikami (we wszystkich zakładkach).<br>
Wszystkie parametry można zmienić w arkuszu (liczba wymaganych pracowników, ich limity czasu pracy).<br>
Wynik generowany jest do arkusza jako nowa zakładka (rozwiązanie skomplikowane) lub do nowego pliku (rozwiązanie proste).<br>
<center>Aleksander Piasecki 215321<br>
17-19, 24 listopada 2020</center>